Name  : Aayush Shah


email : 2001aayushshah@gmail.com


Contact : +91 8879090901

### Task
Build a text classifier using a pre-trained transformer model (such as BERT, RoBERTa, or DistilBERT) that can accurately categorize user queries into following two classes:

● Calendar-related queries

● Gmail-related queries


### Dataset Generation Overview

<a href="https://ibb.co/zW4SHcwZ"><img src="https://i.ibb.co/vCjJv58k/Data-Generation-Full.png" alt="Data-Generation-Full" border="0"></a>

### Summary 
Accuracy : 100%

Overview of Result and Result Generation

<a href="https://ibb.co/PZtV4SXG"><img src="https://i.ibb.co/0jQzZw4y/Calendar-Prediction-Example.png" alt="Calendar-Prediction-Example" border="0"></a>
<a href="https://ibb.co/99fQHt7w"><img src="https://i.ibb.co/ccp4hLjb/Calendar-Prediction-Example2.png" alt="Calendar-Prediction-Example2" border="0"></a>

<a href="https://ibb.co/jPZmtZZK"><img src="https://i.ibb.co/7dNPhNNc/email-Example-Pred.png" alt="email-Example-Pred" border="0"></a>
<a href="https://ibb.co/Fb51yXqB"><img src="https://i.ibb.co/wNSqP7rK/gmail-Search-Result.png" alt="gmail-Search-Result" border="0"></a>

# Dataset Generation for Email and Calendar Queries

* **Data Scarcity Challenge:** Readily available datasets for email and calendar queries were lacking.
* **Significant Time Investment:** 60% of project time spent on dataset generation.
* **Dataset Size Goals:**
    * 500+ data points per label (email/calendar) for initial fine-tuning.
    * 4000+ diverse data points per label for production readiness.
* **Successful Calendar Query Generation:**
    * Over **5000 diverse calendar** queries generated (~10% ambiguous).
    * Utilized structured subtopics  e.g. [ Meeting invites ,Appointments,Reminders , Birthdays] etc. (see `calendarSeeds.txt` for full list).
* **Email Query Strategy:**
    * Defined key subtopics e.g. [Sender/ Recipient,Time/ Date,Keywords/ Subject] (see `emailSeeds.txt` for full list).
    * Leveraged `mistral:7b` model for generation.
* **Prompt Engineering:**
    * Tried out various prompts and parameter finetuning and settled on the below paramters as ideal
        * "system": """You are a highly creative expert at crafting realistic, natural, and diverse user queries. You understand how people speak and can simulate both straightforward and ambiguous calendar-related requests."""
        * temperature values experimented : [0.2,0.3,0.6.0.8]
            * **Observation** : 
                * When temperature= 0.8 i.e. when we asked our model to be generate more creatively , The results were not ideal . 
                * The queries generated were not more Human and were large and not centered around the context given.
                * **Therfore experimenting other temperature values I at last resorted to 0.2 for our query-dataset generation.**
* **Technology Stack:**
    * **locally hosted** `mistral:7b` instruction-finetuned model chosen for instruction following and short queries.
        * **Why Mistral ?** : 
            * The input queries we wish to generate are very short lengthed . 
            * Also Mistral outperforms llama , gemma  and various other open -sourced LLM models for 'instruction follwing tasks ( like ours - generate queries around ...)'
    * Ollama used for local experimentation and seed-based generation.
        * **Why ollama ?**
            * I wanted to leverage Kaggle based resources for generation fo queries from Event dataset 
                instead of wiating for those queries to be generated ,Parallely I was able to generate more queries by leveraging my local resources .

    * Kaggle GPU (P100) utilized for large-scale generation from event data.
* **Time Efficiency:**
    * 3-4 hours for local seed-based query generation (both types).
    * 3 hours for calendar query generation (5113) on Kaggle.
* **Dataset Summary:**
    * **Calendar:** 5113 (Kaggle) + 178 (Local) = **5291 Total**
    * **Email:** 527 (Local)
* **Balanced Training Dataset:** 527 email queries and 527 calendar queries selected for initial training.

### Code - Seed Based Email Query Dataset based generation 

- using Ollama based locally hosted Mistral:7b model

Time Taken : 3-3.5 hours

In [ ]:
import csv
import requests
import time

#Define PARAMETERS
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "mistral:latest"

def create_prompt(seed_topic):
    """
    Creates a prompt to generate email-based user queries based on a given seed topic.

    Args:
        seed_topic (str): A string describing the category of email queries
                          (e.g., "Sender/Recipient: Queries focusing on who sent or received the email.").

    Returns:
        str: A prompt designed to elicit email-based queries related to the seed topic.
    """
    prompt = f"""Based on the following category: 
    {seed_topic}

Generate 25 short and plain natural language queries a user might say when trying to find emails related to this category.

Make sure the queries are:
- Simple, direct, and natural
- Do NOT include phrases like 'Hey Assistant', 'Can you', or 'Please'
- Written as if someone is typing a quick request to a virtual assistant or search bar

Respond with a numbered list of 3 distinct queries only. No explanation or extra text."""
    return prompt


def query_ollama(prompt, temperature=0.3):
    print(prompt)
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "system": """You are a highly creative expert at crafting realistic, natural, and diverse user queries. You understand how people speak when searching or interacting with their email and can simulate both straightforward and ambiguous email-related requests.""",
        "temperature": temperature,
        "stream": False
    }
    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()
    return response.json()["response"]


def parse_response(text):
    queries = []
    for line in text.strip().split("\n"):
        if "." in line:
            parts = line.split(".", 1)
            query = parts[1].strip()
            if query:
                queries.append(query)
    return queries


import csv
import time



def main(seed_file, output_csv):
    seeds = read_seeds(seed_file)
    all_queries = []

    for seed in seeds:
        print(f"Generating queries for: {seed}")
        prompt = create_prompt(seed)
        try:
            response = query_ollama(prompt)
            queries = parse_queries(response)
            all_queries.extend([(query, seed) for query in queries])
            time.sleep(1)  # To avoid hammering the local model
        except Exception as e:
            print(f"Error generating queries for seed '{seed}': {e}")

    print(f"Saving {len(all_queries)} queries to {output_csv}")
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["query", "seed"])
        writer.writerows(all_queries)

In [ ]:
#main function call to generate email_queries from seeds
main("./emailSeeds.txt", "email_queries.csv")

### Code - Calendar Queries Generation - Pre seed based

- Ollama based locally  hosted mistral:7b

In [ ]:
import requests
import csv
import time

OLLAMA_URL = "http://localhost:11434/api/generate"
# MODEL_NAME = "gemma3:1b"
MODEL_NAME = "mistral:latest"


In [ ]:

def read_seeds(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file.readlines() if line.strip()]

def create_prompt(seed):
    return (
        f"Generate about 10  overall short to medium-length natural language queries that a user might say to a virtual assistant. "
        f"Each query should relate to the calendar event type: \"{seed}\".\n"
        f"Queries should include:\n"
        f"- Checking upcoming or past events\n"
        f"- Creating or setting an event\n"
        f"- Cancelling or editing an event\n"
        f"- Asking about event details (time, participants, location, etc.)\n"
        f"- Asking if they are free during a certain time\n"
        f"Respond with a non numbered list, no extra explanation."
    )

def query_ollama(prompt):
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False
    }
    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()
    return response.json()["response"]

def parse_queries(response_text):
    lines = response_text.strip().split("\n")
    queries = []
    for line in lines:
        if "." in line:
            parts = line.split(".", 1)
            query = parts[1].strip()
            if query:
                queries.append(query)
    return queries

def main(seed_file, output_csv):
    seeds = read_seeds(seed_file)
    all_queries = []

    for seed in seeds:
        print(f"Generating queries for: {seed}")
        prompt = create_prompt(seed)
        try:
            response = query_ollama(prompt)
            queries = parse_queries(response)
            all_queries.extend([(query, seed) for query in queries])
            time.sleep(1)  # To avoid hammering the local model
        except Exception as e:
            print(f"Error generating queries for seed '{seed}': {e}")

    print(f"Saving {len(all_queries)} queries to {output_csv}")
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["query", "seed"])
        writer.writerows(all_queries)


In [ ]:
#Testing output  out for one of the seed
prompt = create_prompt("Meeting invites")
response =query_ollama(prompt)

In [ ]:
#Display of the prompt being used
print(prompt)

Generate about 2  overall short to medium-length natural language queries that a user might say to a virtual assistant. Each query should relate to the calendar event type: "Meeting invites".
Queries should include:
- Checking upcoming or past events
- Creating or setting an event
- Cancelling or editing an event
- Asking about event details (time, participants, location, etc.)
- Asking if they are free during a certain time
Respond with a non numbered list, no extra explanation.


In [ ]:
#Response generated by Mistral LLM
print(response)

1. "What meeting invites do I have next week?"
2. "Have I received any new meeting invites today?"
3. "Can you create a meeting invite for next Friday at 3 PM?"
4. "How can I cancel the meeting invite for the 10th of this month?"
5. "Who is attending the meeting on the 15th at 2 PM?"
6. "Where is the meeting scheduled for tomorrow at 11 AM?"
7. "Is there a meeting invite for the same time next week?"
8. "Am I free to take a call at 4 PM today?"
9. "What are the details of the meeting invite on the 20th?"
10. "Can you reschedule the meeting invite from tomorrow to Thursday next week?"


In [ ]:
#Main function call for generation of calendar ueries from pre seeds 
main("./calenderSeeds.txt", "calendar_queries2.csv")

Generating queries for: Anniversaries
Generating queries for: To-dos / Tasks
Generating queries for: All-day events
Generating queries for: Recurring events
Generating queries for: Webinars / Online events
Generating queries for: Holidays (public/federal/company-specific)
Generating queries for: Events with location info (e.g., conferences, offsites)
Generating queries for: Deadlines / Due dates
Generating queries for: Blocked time / Focus time
Generating queries for: Team events (e.g., standups, retrospectives)
Generating queries for: Personal events (e.g., gym, dinner, travel)
Generating queries for: Calendar invites from external people
Generating queries for: Out of Office (OOO) entries
Generating queries for: Time zone changes / cross-timezone meetings
Generating queries for: Tentative / Pending events
Generating queries for: Follow-ups / Post-meeting reminders
Generating queries for: RSVP status-related events (Accepted / Declined / Tentative)
Saving 143 queries to calendar_queri

### Code - Kaggle based Event-data dataset driven calender Query Generation

- Time taken : 3-3.5 hours 
- GPU : P-100
- Run over Kaggle
- **Important:** Upload/ load  the event-dataset before running below code in kaggle
 - Dataset Link : https://www.kaggle.com/datasets/thiyaga158/email-event-extraction-dataset

In [ ]:
# Install necessary libraries
!pip install -q transformers accelerate sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.1 MB/s eta 0:00:00:00:0100:01


In [ ]:
#Setting up hugging face token for using Mistral:7b model
from huggingface_hub import login
# login() # Replace with your  token

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os

#  Hardware detection
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on: {device.upper()}")

#Use fp16 for GPU, fallback to fp32 for CPU
precision = torch.float16 if device == "cuda" else torch.float32

#Load mistral model
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

#Creation of tokenizer - Use same model-tokenizer as that used for generation/ training
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=precision,
    device_map="auto"
)
#Generator Creation which will be used later to generate queries directly 
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Running on: CUDA


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
import random

def create_prompt(event, date_time, venue):
    """
        Logic to create prompt for calendar related query generation from event, date_time , venue extracted from event_dataset
    """
    subcategories = [
        "Scheduling or setting reminders for the event",
        "Asking for event time, date, or location",
        "Modifying, moving, or cancelling the event",
        "Checking for scheduling conflicts",
        "Sending out invites for the event",
        "Follow-ups and post-event reminders",
        "RSVP status-related queries (Accepted / Declined / Tentative)",
        "Recurring event management or queries",
        "Ambiguous queries that could be interpreted as email-related (e.g., invites, follow-ups, sharing info)"
    ]

    selected = random.sample(subcategories, 3)
    selected_text = "\n- " + "\n- ".join(selected)
    system_instruction = """You are a highly creative expert at crafting realistic, natural, and diverse user queries"""
    return (
        f"System :{system_instruction}\n"
        f"Generate 3 short and plain natural language queries a user might say based on the following calendar event:\n"
        f"- Event: {event}\n"
        f"- Date and Time: {date_time}\n"
        f"- Venue: {venue}\n\n"
        f"Each query should relate to a different category randomly chosen from below:{selected_text}\n\n"
        f"Make sure the queries are:\n"
        f"- Simple, direct, and natural\n"
        f"- Do NOT include phrases like 'Hey Assistant', 'Can you', or 'Please'\n"
        f"- Written as if someone is typing a quick request to a virtual assistant or search bar\n\n"
        f"Respond with a numbered list of 3 queries only. No explanation or extra text."
    )


In [ ]:
# Load CSV (uploaded to Kaggle dataset section)
input_path ='/kaggle/input/emailevent/EmailEventExtractionDatasetModified.csv'
df = pd.read_csv(input_path)
# Analysing the loaded dataframe
print(df.head())
print(df['Event'][0])

                          Event                 Date and time  \
0                    SSN Trophy     14th April 2025, 10:30 am   
1  The SSN Best Speaker Contest      4th March 2025, 08:30 am   
2            Science Exhibition     10th March 2025, 10:00 AM   
3                    submission  28th February 2025, 10:00 AM   
4            Annual Sports Meet     15th March 2025, 10:00 AM   

                                        Venue  
0                          new sports complex  
1  GD Room, New First Year Block (CSE annexe)  
2                           school auditorium  
3                                     library  
4                           school playground  
SSN Trophy


In [ ]:

import csv
import time
import os

def main(csv_file, output_csv):
    """
    Processes event data from a CSV to generate user queries and save them.
    """
    processed_events = set()
    
    # Create or load existing output to track processed events
    if os.path.exists(output_csv):
        with open(output_csv, newline='') as out_f:
            reader = csv.reader(out_f)
            next(reader, None)  # skip header
            for row in reader:
                if len(row) > 1:
                    processed_events.add(row[1])  # event name

    with open(csv_file, newline='') as f, open(output_csv, 'a', newline='') as out_f:
        reader = csv.DictReader(f)
        writer = csv.writer(out_f)

        # Write header only if file was empty
        if os.stat(output_csv).st_size == 0:
            writer.writerow(['query', 'event'])
        i=0
        for row in reader:
            # print(row)
            # if i==3:
            #     break
            i = i+1
            print(i)
            event = row['\ufeffEvent']
            if not event or event.strip() == "" or len(event.strip()) < 1:
                continue

            if event in processed_events:
                print(f"Skipping already processed event: {event}")
                continue

            date_time = row['Date and time']
            venue = row['Venue']
            prompt = create_prompt(event, date_time, venue)
            # print(prompt)
            print(f"Generating queries for: {event}")
            try:
                # response = query_ollama(prompt)
                # queries = parse_response(response)
                # queries = generator(prompt, max_new_tokens=256, temperature=0.3)[0]["generated_text"]
                queries = generator(prompt,max_new_tokens=500, temperature=0.3)[0]["generated_text"]
                queries = parse_response(queries)
                
                for q in queries:
                    writer.writerow([q, event])
                    out_f.flush()  # Force write to disk

                # processed_events.add(event)
                # print(queries)
                time.sleep(1)

            except Exception as e:
                print(f"Error generating queries for '{event}': {e}")



In [ ]:
!pwd

/kaggle/input


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
#setup variable to store the generated dataset
output_path = "/kaggle/working/event_queriesFinal.csv"

In [ ]:
# main function call to create queries from event from events dataset
main(input_path , output_path)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1
Skipping already processed event: SSN Trophy
2
Skipping already processed event: The SSN Best Speaker Contest
3
Skipping already processed event: Science Exhibition
4
Generating queries for: submission


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


5
Generating queries for: Annual Sports Meet


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

1260
Generating queries for: Mathematics Olympiad


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1261
Generating queries for: Tech Symposium


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1262
Generating queries for: Robotics Showcase


### I downloaded the generated file event_queriesFinal.csv with 5113 queries generated

## Consolidation and Preprocessing

* **Importance of Action Words:** Analysis of generated data revealed the significance of words like 

["which", "whom", "whose", "why", "can", "should", "would", "could", "do", "did"]  in guiding the action within a query.
* **Custom Stopwords Removal:** A custom list of stopwords was curated, and the generated queries were preprocessed using this list.
* **Lemmatization Eliminated:** After experimentation, **lemmatization was deemed unnecessary**. Transformer-based tokenizers inherently handle word stems (lemmas), minimizing the added benefit of explicit lemmatization.
* **Punctuation and Garbage Token Removal:** Punctuation marks and irrelevant "garbage" tokens were removed from the queries. Importantly, "@" and "." were retained due to their potential relevance in downstream classification tasks.
* **Spell Check Considered:** Using TextBlob
* **Consistent Preprocessing:** The same preprocessing steps (including potential future spell check integration) will be applied to the test data and incoming user queries before model prediction.

In [ ]:
import pandas as pd

# Load both CSV files
email_df = pd.read_csv("emailFinal.csv")
calendar_df = pd.read_csv("calendarFinal.csv")

# Assign labels
email_df["label"] = 0
calendar_df["label"] = 1

# Keep only required columns
email_df = email_df[["Query", "label"]]
calendar_df = calendar_df[["Query", "label"]]

# Concatenate both datasets
combined_df = pd.concat([email_df, calendar_df], ignore_index=True)

# Save to data.csv
combined_df.to_csv("data.csv", index=False)

## Preprocessing

In [ ]:
!pip install nltk textblob

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/624.3 kB ? eta -:--:--
   --------------------------------- ------ 524.3/624.3 kB 1.0 MB/s eta 0:00:01
   ---------------------------------------- 624.3/624.3 kB 1.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import nltk

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()
def preprocess_query(text):
    if not isinstance(text, str):
        return ""

    # 1. Lowercase
    text = text.lower()

    # 2. Spell correction using TextBlob
    text = str(TextBlob(text).correct())

    # 3. Remove punctuation except '@'
    text = re.sub(r"[^\w\s@.]", "", text)

    # 4. Remove stop words
    words = text.split()
    

    filtered_words = [word for word in words if word not in stop_words]

    # 5. Join and strip
    text = " ".join(filtered_words).strip()

    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
data_df = pd.read_csv("data.csv")
data_df = data_df[700 : 800]
# Apply preprocessing
data_df["Query"] = data_df["Query"].apply(preprocess_query)
data_df.to_csv("data_preprocessed.csv", index=False)

### Analysis of Data

In [ ]:
import pandas as pd

# Load your CSV (replace 'data.csv' with your file)
df = pd.read_csv('data.csv')

# Calculate word counts for each query
df['word_count'] = df['Query'].apply(lambda x: len(str(x).split()))

# Max number of words in any query
max_words = df['word_count'].max()

# Average number of words across all queries
avg_words = df['word_count'].mean()

print(f"Max number of words in Query: {max_words}")
print(f"Average number of words in Query: {avg_words:.2f}")


In [ ]:
print(df['label'].value_counts())

In [ ]:
print(df['word_count'].describe())

In [ ]:
print(df.isnull().sum())

In [ ]:
tokens_lengths = df['Query'].apply(lambda x: len(tokenizer.tokenize(str(x))))
print(tokens_lengths.describe())

### Based on the above analysis

* **MAX_LEN = 32:** Maximum query length is 24 tokens; setting `MAX_LEN` to 32 provides sufficient padding and accommodates potential future longer queries.
* **Dataset Length (~1000):** With a relatively small dataset size, a moderate `MAX_LEN` helps prevent excessive padding and reduces computational cost.

### Hyperparameter Choices Explained

* **MODEL_NAME = "microsoft/deberta-v3-base":** DeBERTa-v3-base is a strong general-purpose transformer model known for its performance in various NLP tasks, providing a good balance of accuracy and efficiency for query classification.
* **MAX_LEN = 32:** Based on the analysis that the maximum query length is 24 words, a `MAX_LEN` of 32 provides sufficient padding without excessive computational overhead.
* **BATCH_SIZE = 16:** A `BATCH_SIZE` of 16 offers a good compromise between training stability and speed, fitting within typical memory constraints while allowing for parallel processing.
* **EPOCHS = 4:** Training for 4 epochs is a reasonable starting point to allow the model to learn the patterns in the data without overfitting, especially with a moderately sized dataset.
* **LEARNING_RATE = 2e-5:** A learning rate of 2e-5 is a common and often effective value for fine-tuning pre-trained transformer models, allowing for gradual adaptation to the specific task.

# Query Classification with DeBERTa V3 + MLflow - Training

### Training Details
* **MODEL:** DeBERTa-v3-base
* **Architecture:** GPU P-100 (Kaggle)
* **Framework:** PyTorch, Transformers (Hugging Face library)
* **Transformer Model:** microsoft/deberta-v3-base
* **Tokenizer:** microsoft/deberta-v3-base
* **Loss Metric:** F1 score
* **Inference Metrics:** Precision, F1 score, Recall, Accuracy
* **Logging:** MLflow (Experiment Name: "Query_Classification")
* **Device:** CUDA
* **Post-Prediction Modification:** LLM - Mistral:7b
* **Time Taken:** 3-4 hours

### Why DeBERTa-v3-base?

* **DeBERTa V3 (by Microsoft):** The chosen model is `microsoft/deberta-v3-base`.
* **Improved Contextual Understanding:** It introduces **disentangled attention**, which allows the model to better understand the relationships between words and their positions in a sentence, leading to improved contextual understanding.
* **Strong Classification Performance:** DeBERTa V3 is **ranked among the top** models on many classification benchmarks, indicating its effectiveness for this type of task.
* **Tokenizer: DebertaV2Tokenizer:** The associated tokenizer is `DebertaV2Tokenizer`.
* **Excellent Accuracy:** DeBERTa models often achieve **excellent accuracy**, frequently outperforming RoBERTa on classification tasks.
* Latest in comparsion to other transformer models  : Released in 2021 . compared to Bert , Roberta in 2019

In [ ]:
#installation of required libraries
!pip install -q transformers datasets scikit-learn mlflow sentencepiece

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import mlflow
import mlflow.pytorch

In [ ]:
# Paramter - Configuration 
MODEL_NAME = "microsoft/deberta-v3-base"
MAX_LEN = 32
BATCH_SIZE = 16
EPOCHS = 4
LEARNING_RATE = 2e-5

In [ ]:
# Load dataset
df = pd.read_csv("/kaggle/input/dataprocessed/data_preprocessed.csv")  # <-- Change path if needed
# df.head()
# df = df[:100]

In [ ]:
#Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

### Data preparation for training

In [ ]:
class QueryDataset(Dataset):
    def __init__(self, queries, labels, tokenizer, max_len):
        self.queries = queries
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.queries)

    def __getitem__(self, item):
        query = str(self.queries[item])
        label = int(self.labels[item])

        encoding = self.tokenizer(
            query,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# Split into train-val-test dataset
train_texts, temp_texts, train_labels, temp_labels = train_test_split(df['Query'], df['label'], test_size=0.3, stratify=df['label'], random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42)

train_dataset = QueryDataset(train_texts.values, train_labels.values, tokenizer, MAX_LEN)
val_dataset = QueryDataset(val_texts.values, val_labels.values, tokenizer, MAX_LEN)
test_dataset = QueryDataset(test_texts.values, test_labels.values, tokenizer, MAX_LEN)

In [ ]:
#Loading Base Transformer Model which will be finetuned(trained) on our data
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=LEARNING_RATE,
    logging_dir="./logs",
    do_eval=True,  # Enable evaluation during training
    eval_strategy="epoch",
    save_strategy="epoch",  # Save checkpoints after each epoch
    save_total_limit=3,     # Keep only the last 3 checkpoints
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to=["mlflow"]
)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
#Training the mdoel in sync with MLFLow to log the results against the hyper paramter used
mlflow.set_experiment("Query_Classification")
with mlflow.start_run():
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )
    trainer.train()

    # Evaluate
    eval_result = trainer.evaluate(eval_dataset=test_dataset)
    print("\nTest Evaluation:", eval_result)

    # Log metrics to MLflow
    mlflow.log_metrics(eval_result)
    mlflow.pytorch.log_model(model, "deberta_model_full")

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.063691,0.992908,0.992000,0.984127,1.000000
2,No log,0.008353,1.000000,1.000000,1.000000,1.000000
3,No log,0.001236,1.000000,1.000000,1.000000,1.000000
4,No log,0.001021,1.000000,1.000000,1.000000,1.000000



Test Evaluation: {'eval_loss': 0.0029828951228410006, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 0.3464, 'eval_samples_per_second': 409.959, 'eval_steps_per_second': 25.983, 'epoch': 4.0}


2025/05/15 09:56:36 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/15 09:56:52 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.21.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torchvision==0.21.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/15 09:56:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
#saving the model
model.save_pretrained('/kaggle/working/model/')

### Testing / Inference

In [ ]:
import torch

In [ ]:
from transformers import AutoModelForSequenceClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model = AutoModelForSequenceClassification.from_pretrained('./model/').to(device)

In [ ]:
# preprocess function - the same that was used for training data
import re
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import nltk

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

# stop_words = set(stopwords.words('english'))
retain_words = {
    "what", "how", "when", "where", "who", "which", "whom", "whose", "why",
    "can", "should", "would", "could", "do", "did", "does", "will", "may",
    "show", "find", "search", "get", "have"
}

# Base stopwords from NLTK
default_stop_words = set(stopwords.words("english"))

# Final custom stopword list
custom_stop_words = default_stop_words - retain_words

# lemmatizer = WordNetLemmatizer()
def preprocess_query(text):
    if not isinstance(text, str):
        return ""

    # 1. Lowercase
    text = text.lower()

    # 2. Spell correction using TextBlob
    text = str(TextBlob(text).correct())

    # 3. Remove punctuation except '@'
    text = re.sub(r"[^\w\s@.]", "", text)

    # 4. Remove stop words
    words = text.split()

    filtered_words = [word for word in words if word not in custom_stop_words]

    # 5. Join and strip
    text = " ".join(filtered_words).strip()

    return text

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


### Setting up Mistral LLM for generation of executable query on email / calender

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os


from huggingface_hub import login

# login() # Replace with your actual token

# ⚙️ Step 3: Hardware detection
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on: {device.upper()}")

# ✅ Use fp16 for GPU, fallback to fp32 for CPU
precision = torch.float16 if device == "cuda" else torch.float32

# 🧠 Step 4: Load mistral model
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=precision,
    device_map="auto"
)

generator = pipeline("text-generation", max_new_tokens=256, model=model, tokenizer=tokenizer)

Running on: CUDA


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


Prompts specific to email and calender to generate executable queries

### Here we have used few shot learning to finetune our LLM model to generate executable queries once our transformer model predicts the class of the query

In [ ]:
def generate_email_search_query(email_text):
    """
    Generates an advanced search query using an LLM.

    Args:
        email_text (str): The text of the email.
        classification_label (str): The classification label (e.g., "calendar", "email").

    Returns:
        str: The generated search query.
    """
    #  <----------------------- LLM Interaction --------------------------->
    #  This is the core part where you'd use Mistral.
    #  The prompt should guide Mistral to generate a good search query.
    prompt = f"""
    You are an AI assistant helping a user search their emails.  Generate advanced search queries for Gmail based on the user's request.  Use Gmail's search operators (from:, to:, subject:, has:, etc.) to make the search as precise as possible.  Here are some examples:

Text Query: Find my email from John about the project.
Gmail Advanced Search Query: from:john subject:project

Text Query: Show me the email my boss sent last week.
Gmail Advanced Search Query: from:boss@example.com after:2024/05/08 before:2024/05/15

Text Query: I need the document attached to Mary's email.
Gmail Advanced Search Query: from:mary has:attachment has:document

Text Query: Search for the meeting agenda.
Gmail Advanced Search Query: subject:meeting agenda

Text Query: Find the email where I was copied.
Gmail Advanced Search Query: cc:me@example.com OR bcc:me@example.com

Text Query: Show me the email about the party from Susan before Friday.
Gmail Advanced Search Query: from:susan subject:party before:2024/05/10

Text Query: Find the email with the spreadsheet.
Gmail Advanced Search Query: has:spreadsheet

Text Query: Show me the email from the mailing list.
Gmail Advanced Search Query: list:info@example.org

Text Query: I'm looking for the email with the presentation.
Gmail Advanced Search Query: has:presentation

Text Query: Find the email about the "urgent" report.
Gmail Advanced Search Query: subject:"urgent report"

Text Query: Show me emails from March 6th, 2023 mentioning word 'Google'
Gmail Advanced Search Query: after:2025/05/12 juspay

Text Query: Show me emails from January 11th, 2021 with word 'DRDO' in it
Gmail Advanced Search Query: after:2021/01/11 DRDO

Based  on the above examples generate  Gmail Advanced Search Query for the below Text Query . Answer 1 short search query and nothing else.
Text Query: {email_text}
Gmail Advanced Search Query:"""

    
    response = generator(prompt,max_new_tokens=500, temperature=0.1)[0]["generated_text"]
    return response

In [ ]:

def generate_calendar_search_query(query):
    """
    Generates an advanced search query using an LLM.

    Args:
        email_text (str): The text of the email.
        classification_label (str): The classification label (e.g., "calendar", "email").

    Returns:
        str: The generated search query.
    """
    #  <----------------------- LLM Interaction --------------------------->
    #  This is the core part where you'd use Mistral.
    #  The prompt should guide Mistral to generate a good search query.
    
    prompt = f"""
    You are a helpful assistant designed to generate calendar search queries based on user requests.  Each search query should have two parts:

* **Action:** The action to perform (e.g., "search", "create", "reschedule", "cancel").
* **Search:** The details to use for the search (e.g., "meeting with John", "appointment with Sarah").

Here are some examples of user requests and the corresponding search queries:

Example 1:
User Request: Find my meeting with John.
Search Query: Action: search, Search: meeting with John

Example 2:
User Request: Show me my appointments for tomorrow.
Search Query: Action: search, Search: appointments tomorrow

Example 3:
User Request: When is my next appointment with the doctor?
Search Query: Action: search, Search: next doctor appointment

Example 4:
User Request: Reschedule my meeting with the team to Friday.
Search Query: Action: reschedule, Search: team meeting Friday

Example 5:
User Request: Find the meeting about the project on June 10th.
Search Query: Action: search, Search: meeting about the project June 10th

Example 6:
User Request: Show me all my meetings next week.
Search Query: Action: search, Search: meetings next week

Example 7:
User Request: Cancel my appointment with Sarah.
Search Query: Action: cancel, Search: appointment Sarah

Example 8:
User Request: Find the event scheduled by the "XYZ project group".
Search Query: Action: search, Search: event scheduled by XYZ project group

Example 9:
User Request: Show me all my events in the month of December.
Search Query: Action: search, Search: events December

Example 10:
User Request: Find the event called "conference" that occurs in the next 3 days
Search Query: Action: search, Search: conference next 3 days

Example 11:
User Request: Create a meeting with John tomorrow at 2pm
Search Query: Action: create, Search: meeting with John tomorrow at 2pm

Example 12:
User Request: Update my meeting with John to next Monday
Search Query: Action: update, Search: meeting with John to next Monday

Based  on the above examples generate Search Query for the below Text Query . Answer 1 short search query and nothing else.
Text Query: {query}
Calendar Search Query:"""
    response = generator(prompt,max_new_tokens=500, temperature=0.1)[0]["generated_text"]
    return response

In [ ]:
import re

def extract_after_last_search_query(text):
    """_summary_
        Function to retrieve generated executable query from LLM
    Args:
        text (_type_): _description_

    Returns:
        _type_: _description_
    """
    pattern = r"Search Query:"
    matches = list(re.finditer(pattern, text))

    if matches:
        last_match = matches[-1]
        return text[last_match.end():].strip()
    else:
        return None

### Below are functions that can directly be used to generate results/ predictions

**Please refer to the Function description for usage  and examples below**

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
def get_prediction(text, model, tokenizer, max_len, device='cuda',gen_task_query=False):
    """
    Predicts the sentiment of a given text using the trained model.

    Args:
        text (str): The input text to predict the sentiment for.
        model (torch.nn.Module): The trained PyTorch model.
        tokenizer (transformers.PreTrainedTokenizer): The tokenizer used for the model.
        max_len (int): The maximum sequence length.
        device (str, optional): The device to use ('cuda' or 'cpu'). Defaults to 'cuda'.
        gen_task_query(optional )=If true it will generate executable query via Mistral LLM model

    Returns:
        dict: A dictionary containing the predicted sentiment ('Positive' or 'Negative')
              and its corresponding probability.
    """
    user_query = text
    text = preprocess_query(text)  # Apply the preprocessing here
    model.eval()  # Set the model to evaluation mode
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_len,
        return_tensors='pt'
    )
    encoding = {k: v.to(device) for k, v in encoding.items()}

    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**encoding)
        logits = outputs.logits

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu()).numpy()  # Get probabilities, move to CPU, convert to numpy

    label = np.argmax(probs, axis=-1)
    task_query = ""
    # gen_task_query = True
    if(gen_task_query == True):
        if label == 1:
            task_query = generate_calendar_search_query(user_query)
            task_query = extract_after_last_search_query(task_query)
        else:
            task_query = generate_email_search_query(user_query)
            task_query = extract_after_last_search_query(task_query)
    
    if label == 1:
        return {
            'prediction': 'Calendar',
            'probability': probs[1],
            'task_query' : task_query
        }
    else:
        return {
            'prediction': 'Email',
            'probability': probs[0],
            'task_query' : task_query
        }

def display_predictions(model, test_dataset, tokenizer, max_len, device='cuda'):
    """
    Displays the predictions for the queries in the test set.

    Args:
        model (torch.nn.Module): The trained PyTorch model.
        test_dataset (torch.utils.data.Dataset): The test dataset.
        tokenizer (transformers.PreTrainedTokenizer): The tokenizer used for the model.
        max_len (int): The maximum sequence length.
        device (str, optional): The device to use ('cuda' or 'cpu'). Defaults to 'cuda'.
    """
    model.to(device)  # Ensure model is on the correct device
    all_preds = []
    all_labels = []

    for i in range(len(test_dataset)):
        # if i == 3:
        #     break
        sample = test_dataset[i]
        text = test_dataset.queries[i]  # Access the original text from the dataset
        prediction = get_prediction(text, model, tokenizer, max_len, device)
        ground_truth = test_dataset.labels[i]  # Access the ground truth label.
        all_labels.append(ground_truth)

        if prediction['prediction'] == 'Calendar':
            all_preds.append(1)
        else:
            all_preds.append(0)
        print(f"Query: {text}")
        print(f"Predicted Sentiment: {prediction['prediction']}")
        print(f"Probability: {prediction['probability']:.4f}")  # Format probability
        print(f"Ground Truth: {ground_truth}")
        print("-" * 20)
    
    # Calculate and print metrics after the loop
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    print(f"Overall Accuracy: {accuracy:.4f}")
    print(f"Overall Precision: {precision:.4f}")
    print(f"Overall Recall: {recall:.4f}")
    print(f"Overall F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)

In [ ]:
#Creation of model and tokenizer 
model = AutoModelForSequenceClassification.from_pretrained('/kaggle/working/model/').to('cuda') # Or 'cpu'
    # 2.  Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base", use_fast=False)

In [ ]:
# Function for generating predictions for test_dataset passed
# 0 -> Email
# 1 -> Calendar
display_predictions(model, test_dataset, tokenizer, MAX_LEN)

Query: find email look like automatic announcements
Predicted Sentiment: Email
Probability: 0.9691
Ground Truth: 0
--------------------
Query: show newsletters ive saved reading older
Predicted Sentiment: Email
Probability: 0.9682
Ground Truth: 0
--------------------
Query: march 2025 1000 seminar hall conflict
Predicted Sentiment: Calendar
Probability: 0.9235
Ground Truth: 1
--------------------
Query: can check theres stand schedule tomorrow 3 pm
Predicted Sentiment: Calendar
Probability: 0.9199
Ground Truth: 1
--------------------
Query: find unreal email have attachments.
Predicted Sentiment: Email
Probability: 0.9703
Ground Truth: 0
--------------------
Query: show email subject line containing specific symbol e.g. .
Predicted Sentiment: Email
Probability: 0.9684
Ground Truth: 0
--------------------
Query: company holidays coming next week
Predicted Sentiment: Calendar
Probability: 0.8504
Ground Truth: 1
--------------------
Query: can get reminder meg quit feb 2025
Predicted Sent

### Below are examples of generating individual predictions with ot without final executable query

- Below we are generating task_query via Mistral LLM

In [ ]:
print(get_prediction("What are my upcoming conferences or offsite events?", model, tokenizer, MAX_LEN,'cuda',True))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'prediction': 'Calendar', 'probability': 0.9089517, 'task_query': 'Action: search, Search: upcoming conferences or offsite events'}


In [ ]:
print(get_prediction("cancel birthday party schedule tomorrow", model, tokenizer, MAX_LEN,'cuda',True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'prediction': 'Calendar', 'probability': 0.91840917, 'task_query': 'Action: cancel, Search: birthday party schedule tomorrow'}


- Below we are **not** generating task_query setting **gen_task_query = False**

In [ ]:
print(get_prediction("What are my upcoming conferences or offsite events?", model, tokenizer, MAX_LEN,'cuda',False))

{'prediction': 'Calendar', 'probability': 0.9089517, 'task_query': ''}


- Below we are generating task_query setting **gen_task_query = True**

In [ ]:
print(get_prediction("Show me emails from May 8th, 2025 mentioning 'juspay'", model, tokenizer, MAX_LEN,'cuda',True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'prediction': 'Email', 'probability': 0.9643836, 'task_query': 'after:2025/05/08 juspay'}


## Below I am executing the generated task_query in gmail


<a href="https://ibb.co/Fb51yXqB"><img src="https://i.ibb.co/wNSqP7rK/gmail-Search-Result.png" alt="gmail-Search-Result" border="0"></a>

### End of file